# Freight Pricing API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [1]:
signal_ocean_api_key = 'NotValid' replace with your subscription key

## Calculating a freight price
The Freight pricing API Retrieves freight prices for moving commodities between two port.
The following function is available:

1. get_freight_pricing.

Args:

    vessel_type: The type of vessel to calculate the prices for.
    load_port: Port where the commodity is loaded.
    discharge_port: Port where the commodity is discharged.
    date: Date at which the freight price is requested.
    vessel_subclass: The vessel's subclass.

Returns:

    A collection of freight pricing items, one per vessel class.
    
### Moving commodity from Fujairah towards Rotterdam
The following example will display how to calculate the available freight prices to move a commodity from Fujairah to Rotterdam.
We will use also the PortAPI to map ports to the Signal Geo schema.

In [3]:
from signal_ocean import Connection
connection = Connection(api_key=signal_ocean_api_key)

In [4]:
from signal_ocean.freight_pricing import FreightPricingAPI

freight_pricing_api = FreightPricingAPI(connection)

freight_pricing_api.get_ports()
freight_pricing_api.get_vessel_classes()
freight_pricing_api.get_vessel_types()

(VesselType(id=0, name='Empty'),
 VesselType(id=1, name='Tanker'),
 VesselType(id=3, name='Dry'),
 VesselType(id=4, name='Container'),
 VesselType(id=5, name='Lng'),
 VesselType(id=6, name='Lpg'),
 VesselType(id=-2, name='NotSet'),
 VesselType(id=-1, name='Unknown'))

In [5]:
from signal_ocean.freight_pricing import FreightPricingAPI, VesselTypeFilter, VesselClassFilter, PortFilter

freight_pricing_api.get_ports(PortFilter(name_like='roa'))
freight_pricing_api.get_vessel_classes(VesselClassFilter(name_like='max'))
freight_pricing_api.get_vessel_types(VesselTypeFilter(name_like='Tank'))

(VesselType(id=1, name='Tanker'),)

### Call the freight pricing API
Now that we have the ports we want to query about, we can call the freight pricing API and get out the commodity transfer cost.

In [6]:
from signal_ocean.freight_pricing import FreightPricingAPI,VesselType,VesselSubclass
from datetime import date

freight_pricing_api = FreightPricingAPI()

vessel_type = freight_pricing_api.get_vessel_types(VesselTypeFilter(name_like='tanker'))[0]
load_port = freight_pricing_api.get_ports(PortFilter(name_like='Fujairah'))[0]
discharge_port = freight_pricing_api.get_ports(PortFilter(name_like='Rotterdam'))[0]

today = date.today()

freight_pricing = freight_pricing_api.get_freight_pricing(
    vessel_type,
    load_port=load_port, 
    discharge_port=discharge_port,
    date=today,
    vessel_subclass=VesselSubclass.DIRTY
)

### Print out the result

Now we can hold the result and print it out on a dataframe

In [7]:
import pandas as pd

dataf = pd.DataFrame([{
    'vessel_class': p.vessel_class,
    'cargo_quantity': p.cargo_quantity,
    **p.costs.__dict__,
    **p.totals.__dict__ } for p in freight_pricing
])

dataf

,vessel_class,cargo_quantity,freight_rate,freight_cost,canal,total_cost,total_cost_per_ton
0,VLCC,280000.0,7.98664,2236259.2,0.0,2236259.2,7.98664
1,Suezmax,140000.0,-2.53416,-354782.4,713288.0,358505.6,2.5607542857142858
2,Aframax,80000.0,13.1875,1055000.0,599610.0,1654610.0,20.682625
